In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Looking at the data

In [ ]:
data = pd.read_csv("../input/Hotel_Reviews.csv")

In [ ]:
data.head()

In [ ]:
for i in data.columns:
    print(i)

In [ ]:
len(data.Hotel_Name.unique()) #total number of distinct hotels

In [ ]:
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 50, 18
rcParams["axes.labelsize"] = 16
from pandas import read_csv
from pandas import datetime
from matplotlib import pyplot
import seaborn as sns

In [ ]:
data_plot = data[["Hotel_Name","Average_Score"]].drop_duplicates()
sns.set(font_scale = 2.5)
a4_dims = (30, 12)
fig, ax = pyplot.subplots(figsize=a4_dims)
sns.countplot(ax = ax,x = "Average_Score",data=data_plot)

We can see that most of the reviews lie in the range of 8.1 to 9.1

In [ ]:
text = ""
for i in range(data.shape[0]):
    text = " ".join([text,data["Reviewer_Nationality"].values[i]])

In [ ]:
from wordcloud import WordCloud
wordcloud = WordCloud(background_color='white', width=600, height=300, max_font_size=50, max_words=40).generate(text)
wordcloud.recolor(random_state=312)
plt.imshow(wordcloud)
plt.title("Wordcloud for countries ")
plt.axis("off")
plt.show()

It seems like there are a lot of people from United Kingdom here.

In [ ]:
len(data.Hotel_Name.unique())

In [ ]:
data["pos_count"] = 1
data["neg_count"] = 1

In [ ]:
data["pos_count"] = data.apply(lambda x: 0 if x["Positive_Review"] == 'No Positive' else x["pos_count"],axis =1)

In [ ]:
data["pos_count"].value_counts()

In [ ]:
data["neg_count"] = data.apply(lambda x: 0 if x["Negative_Review"] == 'No Negative' else x["neg_count"],axis =1)

In [ ]:
data["neg_count"].value_counts()

In [ ]:
reviews = pd.DataFrame(data.groupby(["Hotel_Name"])["pos_count","neg_count"].sum())

In [ ]:
# reviews.head()
reviews["Hotel_Name"] = reviews.index
reviews.index = range(reviews.shape[0])

In [ ]:
reviews.head()

In [ ]:
reviews["total"] = reviews["pos_count"] + reviews["neg_count"]

In [ ]:
data["count"] = 1
count_review = data.groupby("Hotel_Name",as_index=False)["count"].sum()

Now we will look at top 10 hotels in europe based upon their popularity. Popularity of a hotel can be determined by the total number of reviews the hotel has. Following is the list of top 10 hotels.

In [ ]:
reviews = pd.merge(reviews,count_review,on = "Hotel_Name",how = "left")

In [ ]:
reviews.head()

In [ ]:
for i in reviews.sort_values(by = "count",ascending=False)["Hotel_Name"].head(10).values:
    print(i)

In [ ]:
reviews["pos_ratio"] = reviews["pos_count"].astype("float")/reviews["total"].astype("float")

Among top 100 famous hotels we will now find which of the hotels are top rated. Looking at top 10 famous hotels with positive reviews.

In [ ]:
famous_hotels = reviews.sort_values(by = "count",ascending=False).head(100)

Checking the address of these famous hotels.

In [ ]:
pd.set_option('display.max_colwidth', 2000)
popular = famous_hotels["Hotel_Name"].values[:10]
data.loc[data['Hotel_Name'].isin(popular)][["Hotel_Name","Hotel_Address"]].drop_duplicates()

All the famous hotels are in London. Now, checking the address of positive hotels

In [ ]:
for i in famous_hotels.sort_values(by = "pos_ratio",ascending=False)["Hotel_Name"].head(10):
    print(i)

In [ ]:
pos = famous_hotels.sort_values(by = "pos_ratio",ascending=False)["Hotel_Name"].head(10).values
data.loc[data['Hotel_Name'].isin(pos)][["Hotel_Name","Hotel_Address"]].drop_duplicates()

**Findings till now**
* Most of the ratings for hotels are between 8.1 to 9.1
* Most of the people are from United Kingdom.

* Top 10 famous Hotels in Europe are(on the basis of number of reviews):
         * Britannia International Hotel Canary Wharf - London 
         * Strand Palace Hotel - London
         * Park Plaza Westminster Bridge London - London
         * Copthorne Tara Hotel London Kensington - London
         * DoubleTree by Hilton Hotel London Tower of London - London
         * Grand Royale London Hyde Park - London
         * Holiday Inn London Kensington - London
         * Hilton London Metropole - London
         * Millennium Gloucester Hotel London - London
          * Intercontinental London The O2 - London

* Hotels with most Positive reviews among the famous hotels(on the basis of number of positive reviews):
         * Apex Temple Court Hotel - London
         * Shangri La Hotel at The Shard London -  London
         * Club Quarters Hotel Lincoln s Inn Fields - London
         * The Piccadilly London West End - London
         * Urban Lodge Hotel  - Netherlands
         * Apex City Of London Hotel - London
         * Hotel Esther a - Netherlands
         * Hilton London Canary Wharf - London
         * Hotel Berna - Italy
         * The Student Hotel Amsterdam City - Netherlands


Now, we will look into the hotels which have maintained most consistent performance in terms of overall rating

In [ ]:
data.Review_Date = pd.to_datetime(data.Review_Date)

In [ ]:
temp = data.groupby("Hotel_Name", as_index=False)["Reviewer_Score"].agg([np.mean, np.std]).sort_values("mean",ascending=False)
temp = temp[temp["mean"] > 8.9]
temp.shape
temp.sort_values("std").index[0:20]

In [ ]:
lis = ['H10 Casa Mimosa 4 Sup', 'Hotel Casa Camper',
       'H tel de La Tamise Esprit de France', 'Le Narcisse Blanc Spa',
       'Hotel Eiffel Blomet', '45 Park Lane Dorchester Collection', '41',
       'Hotel Stendhal Place Vend me Paris MGallery by Sofitel',
       'H tel D Aubusson', 'Hotel The Serras', 'Hotel Am Stephansplatz',
       'Lansbury Heritage Hotel', 'Covent Garden Hotel', 'The Soho Hotel',
       'Catalonia Magdalenes', 'H tel Saint Paul Rive Gauche',
       'Milestone Hotel Kensington', 'Ritz Paris', 'H tel Fabric',
       'Le 123 S bastopol Astotel']
data.loc[data['Hotel_Name'].isin(lis)][["Hotel_Name","Hotel_Address","Average_Score"]].drop_duplicates()

**THE MOST CONSISTENT HOTELS**

    H10 Casa Mimosa 4 Sup - Rating : 9.6, Location : Barcelona, Spain

    Hotel Casa Camper  - Rating : 9.6, Location : Barcelona, Spain

    H tel de La Tamise Esprit de France  - Rating : 9.6, Location : Paris, France

    Le Narcisse Blanc Spa - Rating : 9.5, Location : Paris, France

    Hotel Eiffel Blomet  - Rating : 9.4, Location : Paris, France

    45 Park Lane Dorchester Collection - Rating : 9.4 , Location : London, UK 

    41 - Rating : 9.6 , Location : London, UK  

    Hotel Stendhal Place Vend me Paris MGallery by Sofitel   - Rating : 8.8, Location : Paris, France

    H tel D Aubusson - Rating : 9.4, Location : Paris, France

    Hotel The Serras - Rating : 9.6, Location : Barcelona, Spain

    Hotel Am Stephansplatz  - Rating : 9.3, Location : Vienna, Austria

    Lansbury Heritage Hotel  - Rating : 9.4 , Location : London, UK 

    Covent Garden Hotel   - Rating : 9.4 , Location : London, UK 

    The Soho Hotel   - Rating : 9.5 , Location : London, UK 

    Catalonia Magdalenes  - Rating : 9.6, Location : Barcelona, Spain

    H tel Saint Paul Rive Gauche  - Rating : 8.9, Location : Paris, France

    Milestone Hotel Kensington  - Rating : 9.5 , Location : London, UK 

    Ritz Paris - Rating : 9.8, Location : Paris, France

    H tel Fabric - Rating : 9.3, Location : Paris, France

    Le 123 S bastopol Astotel - Rating : 9.3, Location : Paris, France